In [1]:
mutable struct WEdge
    u::Int
    v::Int
    w::Float64
end

mutable struct WGraph
    adj::Dict{Int, Vector{WEdge}}
end

function addedge!(g::WGraph, u::Int, v::Int, w::Float64)
    if !haskey(g.adj, u)
        g.adj[u] = Vector{WEdge}()
    end
    if !haskey(g.adj, v)
        g.adj[v] = Vector{WEdge}()
    end

    push!(g.adj[u], WEdge(u, v, w))
end

# Init single source function
function initsinglesource(graph::WGraph, s::Int)
    dist = Dict{Int, Float64}()
    pred = Dict{Int, Union{Int, Nothing}}()
    for v in keys(graph.adj)
        dist[v] = Inf
        pred[v] = nothing
    end
    dist[s] = 0.0

    return dist, pred
end


# Relaxing function
function relax!(dist::Dict{Int, Float64}, pred::Dict{Int, Union{Int, Nothing}}, u::Int, v::Int, w::Float64)
    if dist[v] > dist[u] + w
        dist[v] = dist[u] + w
        pred[v] = u
    end
end

# Bellman-Ford algorithm
function bellmanford(graph::WGraph, s::Int)
    dist, pred = initsinglesource(graph, s)
    V = keys(graph.adj)

    for _ in 1:(length(V) - 1)
        for u in V
            for edge in graph.adj[u]
                relax!(dist, pred, edge.u, edge.v, edge.w)
            end
        end
    end

    # Check for negative-weight cycles
    for u in V
        for edge in graph.adj[u]
            if dist[edge.v] > dist[edge.u] + edge.w
                return false, dist, pred
            end
        end
    end

    return true, dist, pred
end

bellmanford (generic function with 1 method)

In [2]:
using Random
using BenchmarkTools
using Statistics

function generate_weighted_graph(n::Int, edge_prob::Float64 = 0.3)
    g = WGraph(Dict{Int, Vector{WEdge}}())
    for u in 1:n
        for v in 1:n
            if u != v && rand() < edge_prob
                w = rand() * 15.0 - 5.0 # (-5 to 10)
                addedge!(g, u, v, w)
            end
        end
    end

    return g
end

function estimate_instances_bf(graph::WGraph, s::Int, target_time::Float64 = 420.0)
    trials = 3
    times = [@elapsed bellmanford(graph, s) for _ in 1:trials]
    avg_time = mean(times)
    est = max(1, Int(round(target_time / avg_time)))
    return est
end

function measure_bf_time(graph::WGraph, s::Int, target_time::Float64 = 420.0)
    instances = estimate_instances_bf(graph, s, target_time)
    times = [@elapsed bellmanford(graph, s) for _ in 1:instances]
    avg_time = mean(times)
    total_time = sum(times)
    return avg_time, total_time, instances, times
end

function run_bellmanford_experiment()
    sizes = [10, 50, 100, 200, 400, 600, 800, 1000]
    all_times = []
    labels = []

    for size in sizes
        println("Measuring Bellman-Ford for size $size")
        g = generate_weighted_graph(size)
        avg, total, instances, inst_times = measure_bf_time(g, 1)
        println("Instances: $instances | Avg time: $(round(avg, digits=4))s | Total time: $(round(total, digits=2))s")

        append!(all_times, inst_times)
        append!(labels, size)
    end

    return labels, all_times
end

run_bellmanford_experiment (generic function with 1 method)

In [ ]:
using StatsPlots

labels, times = run_bellmanford_experiment()

violin(labels, times,
       xlabel = "Graph Size (n)",
       ylabel = "Computation Time (s)",
       title = "Bellman-Ford Runtime Distribution",
       legend = false,
       fillalpha = 0.6)

Measuring Bellman-Ford for size 10
Instances: 28125000 | Avg time: 0.0s | Total time: 298.87s
Measuring Bellman-Ford for size 50
Instances: 252946 | Avg time: 0.0016s | Total time: 411.85s
Measuring Bellman-Ford for size 100
Instances: 37520 | Avg time: 0.0116s | Total time: 433.74s
Measuring Bellman-Ford for size 200
Instances: 4655 | Avg time: 0.0893s | Total time: 415.71s
Measuring Bellman-Ford for size 400
Instances: 432 | Avg time: 0.9441s | Total time: 407.86s
Measuring Bellman-Ford for size 600
